In [1]:
# 필요한 모듈 불러오기

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import numpy as np
import pandas as pd

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action = 'ignore')

import pickle

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Current cuda device:', torch.cuda.current_device())

Current cuda device: 0


In [2]:
# RMSE 준비

def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))


# 모델 준비

## 1. Bestseller
def Biased_Bestseller(train_data, test_data):

    train = train_data.copy()
    test = test_data.copy()

    # 아이템별 평균 평점 계산
    rating_mean = train.groupby('item_id')['rating'].mean()
    test = test.join(rating_mean, on='item_id', rsuffix='_item')

    # 전체 평균 평점 계산
    global_mean = train['rating'].mean()
    test['rating_item'].fillna(train['rating'].mean(), inplace=True)

    # 사용자별 평균 평점
    user_mean = train.groupby('user_id')['rating'].mean()
    test = test.join(user_mean, on='user_id', rsuffix='_user')
    
    
    test['predicted_rating'] = test['rating_item'] - global_mean + test['rating_user']

    rmse_result = RMSE(test['rating'], test['predicted_rating'])

    return rmse_result


## 3. NeuMF

# Dataset 생성
class CustomDataset(Dataset):
    def __init__(self, user_ids, item_ids, ratings, text_embeddings):
        self.user_ids = torch.tensor(user_ids, dtype=torch.long)
        self.item_ids = torch.tensor(item_ids, dtype=torch.long)
        self.ratings = torch.tensor(ratings, dtype=torch.float)
        self.text_embeddings = torch.tensor(text_embeddings, dtype=torch.float)

    def __len__(self):
        return len(self.ratings)
    
    def __getitem__(self, idx):
        return self.user_ids[idx], self.item_ids[idx], self.ratings[idx], self.text_embeddings[idx]

class NeuMF(nn.Module):
    def __init__(self, num_users, num_items, num_layers, embedding_size, num_factors, text_emb_size, dropout=None):
        super().__init__()

        self.dropout = dropout

        # FM part
        self.user_embedding_fm = nn.Embedding(num_users, num_factors)
        self.item_embedding_fm = nn.Embedding(num_items, num_factors)

        # self.w_0 = nn.Parameter(torch.zeros(1))  # global bias
        self.w = nn.Parameter(torch.Tensor(num_factors * 2 + text_emb_size))  # 특성별 가중치
        self.v = nn.Parameter(torch.Tensor(num_factors * 2 + text_emb_size, embedding_size))  # 잠재 요인 가중치

        # MLP part
        self.user_embedding_mlp = nn.Embedding(num_users, embedding_size * (2 ** (num_layers - 1)))
        self.item_embedding_mlp = nn.Embedding(num_items, embedding_size * (2 ** (num_layers - 1)))

        mlp_input_size = embedding_size * (2 ** (num_layers - 1)) * 2 + text_emb_size

        layers = []
        for i in range(num_layers):
            output_size = embedding_size * (2 ** (num_layers - i - 1))
            if dropout:
                layers.append(nn.Dropout(p=self.dropout))
            layers.append(nn.Linear(mlp_input_size, output_size))
            layers.append(nn.ReLU())
            mlp_input_size = output_size
        
        self.mlp_layers = nn.Sequential(*layers)

        # Final prediction layer
        # final_layer_input_size = embedding_size + output_size + text_emb_size
        self.final_layer = nn.Linear(222, 1)

        self._init_weight_()

    def _init_weight_(self):
        # Initialize weights here
        nn.init.normal_(self.user_embedding_fm.weight, std=0.01)
        nn.init.normal_(self.item_embedding_fm.weight, std=0.01)
        nn.init.normal_(self.user_embedding_mlp.weight, std=0.01)
        nn.init.normal_(self.item_embedding_mlp.weight, std=0.01)

        nn.init.normal_(self.w, std=0.01)
        nn.init.normal_(self.v, std=0.01)

        for layer in self.mlp_layers:
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_normal_(self.final_layer.weight)
        
        nn.init.kaiming_normal_(self.final_layer.weight)
    
    def forward(self, user_input, item_input, text_input):
        # FM part
        user_emb_mf = self.user_embedding_fm(user_input)
        item_emb_mf = self.item_embedding_fm(item_input)

        x = torch.cat([user_emb_mf, item_emb_mf, text_input], dim=1)
        # print(x.shape)

        # 1차 상호작용: 각 사용자와 아이템의 가중치를 곱하고, 그 결과를 모두 더함
        linear_terms = torch.sum(x * self.w, dim=1)

        # print(linear_terms.shape)

        # 2차 상호작용
        interactions = 0.5 * torch.sum(
            torch.pow(torch.matmul(x, self.v), 2) - torch.matmul(torch.pow(x, 2), torch.pow(self.v, 2)), dim=1)
        
        # print(interactions.shape)


        # 예측값 계산
        # predict = self.w_0 + linear_terms + interactions
        linear_terms = linear_terms.unsqueeze(-1)
        interactions = interactions.unsqueeze(-1)
        predict = torch.cat([linear_terms, interactions], dim=1)

        # pred2 = nn.Linear(2, 220)(predict)

        # predict = predict.unsqueeze(-1)
        # print(predict.shape)


        # MLP part
        user_emb_mlp = self.user_embedding_mlp(user_input)
        item_emb_mlp = self.item_embedding_mlp(item_input)
        mlp_vector = torch.cat([user_emb_mlp, item_emb_mlp, text_input], dim=1)
        mlp_vector = self.mlp_layers(mlp_vector)

        # print(mlp_vector.shape)

        vector = torch.cat([predict, mlp_vector],dim=1)
        # vector = torch.cat([pred2, mlp_vector], dim=1)

        # print(vector.shape)
    

        rating = self.final_layer(vector)
        return rating.squeeze()



class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        """
        Early stops the training if validation loss doesn't improve after a given patience.
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

def train_and_evaluate(model, criterion, optimizer, train_loader, val_loader, epochs, scheduler=None, patience=5):
    early_stopping = EarlyStopping(patience=patience)
    train_rmse_hist = []
    val_rmse_hist = []

    best_val_rmse = float('inf')
    best_epoch = 0
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for user, item, rating, text in train_loader:
            user, item, rating, text = user.to(device), item.to(device), rating.float().to(device), text.float().to(device)
            
            optimizer.zero_grad()
            prediction = model(user, item, text)
            loss = criterion(prediction.view(-1), rating.view(-1))
            # loss = criterion(prediction, rating)
            loss.backward()

            # torch.nn.utils.clip_grad_norm_(model.parameters(), 2)
            
            optimizer.step()
            
            total_loss += loss.item()
        
        if scheduler:
            scheduler.step()
        
        avg_loss = total_loss / len(train_loader)
        train_rmse = torch.sqrt(torch.tensor(avg_loss))
        train_rmse_hist.append(train_rmse.item())

        val_rmse = evaluate(model, criterion, val_loader)
        val_rmse_hist.append(val_rmse.item())

        if val_rmse < best_val_rmse:
            best_val_rmse = val_rmse
            best_epoch = epoch + 1
        
        print(f"Epoch {epoch+1}/{epochs}, Average Loss: {avg_loss:.6f}, Validation RMSE: {val_rmse:.6f}")
        
        early_stopping(val_rmse)
        if early_stopping.early_stop:
            print("Early stopping triggered. Stopping training.")
            break

    print(f"Best Validation RMSE: {best_val_rmse:.8f} at Epoch {best_epoch}")
    return train_rmse_hist, val_rmse_hist, best_val_rmse


def evaluate(model, criterion, val_loader):
    model.eval()
    with torch.no_grad():
        total_loss = 0
        for user, item, rating, text in val_loader:
            user, item, rating, text = user.to(device), item.to(device), rating.float().to(device), text.float().to(device)
            prediction = model(user, item, text)
            loss = criterion(prediction.view(-1), rating.view(-1))
            # loss = criterion(prediction, rating)
            total_loss += loss.item()
    avg_loss = total_loss / len(val_loader)
    return torch.sqrt(torch.tensor(avg_loss))

In [3]:
# 실험 준비

# 데이터 불러오기
ratings = pd.read_csv('/home/ryu/thesis/data/amazon/Amazon_ratings.csv')
reviews = pd.read_csv('/home/ryu/thesis/data/amazon/Amazon_reviews.csv')

ratings = ratings[['item_id', 'user_id', 'rating']]
reviews = reviews[['item_id', 'user_id', 'text']]

## 기본 전처리
cnt = ratings.groupby('user_id').count()['rating']
keys = cnt[cnt>3].keys()
ratings = ratings[ratings['user_id'].isin(keys)]

with open('/home/ryu/thesis/real_amazon/additional_var/sbert_emb.pickle', 'rb') as f:
    embeddings = pickle.load(f)

emb = pd.DataFrame(embeddings)

data = pd.merge(ratings, reviews, how='left', left_on=['user_id', 'item_id'], right_on=['user_id', 'item_id'])
data = pd.concat([data, emb], axis=1)

## train, test set 나누기
x = data.copy()
y = data['user_id']
ratings_train, ratings_test = train_test_split(x, test_size=0.25, stratify=y, random_state=84)

## Black Sheep 사용자 목록 불러오기
with open('/home/ryu/thesis/real_amazon/black_id.pkl', 'rb') as f:
    black = pickle.load(f)

## train, test 에서 black sheep 사용자만 추출
black_train = ratings_train[ratings_train['user_id'].isin(black)]
black_test = ratings_test[ratings_test['user_id'].isin(black)]
black_all = ratings[ratings['user_id'].isin(black)]

## 사용자 수 구하기 (이후 가중평균 위함)
entire_pop = ratings.user_id.nunique()      # 전체 사용자 수
black_pop = len(black)                      # Black Sheep 사용자 수
rest_pop = entire_pop - black_pop           # 전체 - black sheep = white & gray 사용자 수

## black sheep 제거 데이터
ratings_train = ratings_train[~ratings_train['user_id'].isin(black)]
ratings_test = ratings_test[~ratings_test['user_id'].isin(black)]
ratings = ratings[~ratings['user_id'].isin(black)]

## 사용자 ID와 영화 ID를 연속적인 인덱스로 매핑
user_to_index = {user: idx for idx, user in enumerate(ratings["user_id"].unique())}
item_to_index = {item: idx for idx, item in enumerate(ratings["item_id"].unique())}

## NeuMF는 맵핑된 id를 사용하기 때문에 미리 변환
ratings_train['user_id_conv'] = ratings_train.user_id.map(user_to_index)
ratings_train['item_id_conv'] = ratings_train.item_id.map(item_to_index)
ratings_test['user_id_conv'] = ratings_test.user_id.map(user_to_index)
ratings_test['item_id_conv'] = ratings_test.item_id.map(item_to_index)

## White Sheep 대상 MF 실험 결과 불러오기 (NeuMF 실험 시 사용)
with open('/home/ryu/thesis/real_amazon/additional_var/White_FM/84c_White FM.pkl', 'rb') as f:
    white_results_loaded = pickle.load(f)

## Open saved user_gsu_dict (Gray Sheep id 불러오기)
with open('/home/ryu/thesis/real_amazon/additional_var/1_gsu_data/FM_84_cosine_gsu.pkl', 'rb') as f:
    gray_dict = pickle.load(f)

In [4]:
# 실험 시작

## 1. 결과 저장할 딕셔너리 생성

white_fm = {}
bestseller = {}
weighted_bestseller = {}
gray_bs = {}
weighted_gray_bs = {}
gray_fm = {}
weighted_gray_fm = {}
neumf = {}
weighted_neumf = {}

blk_bs = {}
blk_weighted_bestseller = {}
blk_weighted_gray_bs = {}
blk_weighted_gray_fm = {}
blk_weighted_neumf = {}

## 2. Black Sheep 실험 (trained with only black_train set; gray sheep 기준에 따라 변함 없기 때문에 미리 계산)
blk_only_bestseller = Biased_Bestseller(black_train, black_test)
print('*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*')
print(f'Black Bestseller RMSE (trained with only black_train): {blk_only_bestseller}')
print('*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*')

## 3. 기준별로 gray sheep id 가져와서 실험

test_list = [str(n) for n in range(10, 51, 5)]

for thresh in test_list:

    
    gray_idx = gray_dict[thresh]    # thresh%에 해당하는 Gray sheep 사용자 id 가져오기
    white_rmse = white_results_loaded[thresh]    # thresh%에 해당하는 White Sheep MF 결과 가져오기 (NeuMF 실험 시에만 사용!)

    print('**************************************************')
    print(f'                     {thresh}% 실험 시작                ')
    print('**************************************************')


    # white, gray sheep 사용자 분리
    white = ratings[~ratings['user_id'].isin(gray_idx)]
    gray = ratings[ratings['user_id'].isin(gray_idx)]

    white_train = ratings_train[~ratings_train['user_id'].isin(gray_idx)]
    white_test = ratings_test[~ratings_test['user_id'].isin(gray_idx)]

    gray_train = ratings_train[ratings_train['user_id'].isin(gray_idx)]
    gray_test = ratings_test[ratings_test['user_id'].isin(gray_idx)]

    # NeuMF 데이터 준비
    gray_new_idx = []
    for g in gray_idx:
        gray_new_idx.append(user_to_index[g])

    neumf_gray_train = ratings_train[ratings_train['user_id_conv'].isin(gray_new_idx)]
    neumf_gray_test = ratings_test[ratings_test['user_id_conv'].isin(gray_new_idx)]


    #### 0. Black Sheep Bestseller (trained with White Sheep train set)
    blk_bestseller = Bestseller(white_train, black_test)

    print(f'{thresh}% Black Bestseller RMSE (trained with white_train): {blk_bestseller}')
    blk_bs[f'{thresh}'] = blk_bestseller


    # #### 5. Gray NeuMF ####

    # 하이퍼파라미터 설정
    num_users = len(set(data.user_id)) + 1
    num_items = len(set(data.item_id)) + 1
    num_layers=3
    embedding_size = 220
    num_factors = 220
    text_emb_size = 384
    epochs = 50
    batch_size = 32
    patience = 3

    # 데이터 로더 정의 
    train_text_embeddings = emb.iloc[ratings_train['user_id_conv']].values
    test_text_embeddings = emb.iloc[ratings_test['user_id_conv']].values

    train_dataset = CustomDataset(
        user_ids = neumf_gray_train['user_id_conv'].values,
        item_ids = neumf_gray_train['item_id_conv'].values,
        ratings = neumf_gray_train['rating'].values.astype(np.float32),
        text_embeddings = train_text_embeddings
    )
    test_dataset = CustomDataset(
        user_ids = neumf_gray_test['user_id_conv'].values,
        item_ids = neumf_gray_test['item_id_conv'].values,
        ratings = neumf_gray_test['rating'].values.astype(np.float32),
        text_embeddings = test_text_embeddings
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    # 모델 인스턴스 생성
    model = NeuMF(num_users, num_items, num_layers, embedding_size, num_factors, text_emb_size)

    # 손실 함수와 옵티마이저 설정
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.0014, weight_decay=0.015)
    # optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    model.to(device)
    criterion.to(device)

    train_rmse, val_rmse, best_RMSE = train_and_evaluate(model, criterion, optimizer, train_loader, test_loader, epochs, scheduler, patience)

    print(f'{thresh}% NeuMF RMSE: {best_RMSE.item()}')
    neumf[f'{thresh}'] = best_RMSE.item()
    

    # white sheep trained, black sheep tested
    weighted_gray_neumf = (blk_bestseller * (black_pop/entire_pop)) + (white_rmse * ((rest_pop-len(gray_idx))/entire_pop)) + (best_RMSE.item() * (len(gray_idx)/entire_pop))
    print(f'{thresh}% NeuMF weighted RMSE (w/ White Sheep): {weighted_gray_neumf}')
    weighted_neumf[f'{thresh}'] = weighted_gray_neumf

    # black sheep trained, black sheep tested
    blk_weighted_gray_neumf = (blk_only_bestseller * (black_pop/entire_pop)) + (white_rmse * ((rest_pop-len(gray_idx))/entire_pop)) + (best_RMSE.item() * (len(gray_idx)/entire_pop))
    print(f'{thresh}% NeuMF weighted RMSE (w/ Black Sheep): {blk_weighted_gray_neumf}')
    blk_weighted_neumf[f'{thresh}'] = blk_weighted_gray_neumf



    print(f'                {thresh}% 실험 끝                 ')


# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------


*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*
Black Bestseller RMSE (trained with only black_train): 1.2977171777421705
*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*
**************************************************
                     10% 실험 시작                
**************************************************


10% Black Bestseller RMSE (trained with white_train): 1.155138642682192
Epoch 1/50, Average Loss: 3.395843, Validation RMSE: 1.062168
Epoch 2/50, Average Loss: 1.102363, Validation RMSE: 1.060741
Epoch 3/50, Average Loss: 1.101958, Validation RMSE: 1.060612
Epoch 4/50, Average Loss: 1.101910, Validation RMSE: 1.062780
Epoch 5/50, Average Loss: 1.102041, Validation RMSE: 1.060504
Epoch 6/50, Average Loss: 1.101123, Validation RMSE: 1.060430
Epoch 7/50, Average Loss: 1.101147, Validation RMSE: 1.060714
Epoch 8/50, Average Loss: 1.101667, Validation RMSE: 1.060735
Epoch 9/50, Average Loss: 1.100816, Validation RMSE: 1.060324
Epoch 10/50, Average Loss: 1.100241, Validation RMSE: 1.060371
Epoch 11/50, Average Loss: 1.099918, Validation RMSE: 1.060354
Epoch 12/50, Average Loss: 1.100565, Validation RMSE: 1.060318
Epoch 13/50, Average Loss: 1.099190, Validation RMSE: 1.060419
Epoch 14/50, Average Loss: 1.099483, Validation RMSE: 1.060298
Epoch 15/50, Average Loss: 1.099903, Validation RMSE: 1

In [5]:
# 실험 결과 저장
with open('/home/ryu/thesis/real_amazon/additional_var/Gray_NeuMF/84c_NeuMF.pkl', 'wb') as f:
    pickle.dump(neumf, f)
with open('/home/ryu/thesis/real_amazon/additional_var/Gray_NeuMF/84c_Weighted NeuMF.pkl', 'wb') as f:
    pickle.dump(weighted_neumf, f)
with open('/home/ryu/thesis/real_amazon/additional_var/Gray_NeuMF/84c_Weighted NeuMF Black.pkl', 'wb') as f:
    pickle.dump(blk_weighted_neumf, f)


print(f'               실험 결과 저장 완료 😎               ')



               실험 결과 저장 완료 😎               
